In [2]:
import ipyparallel as ipp

In [41]:
rc = ipp.Client(profile="bluefog")
rc.ids

[0, 1, 2, 3]

In [42]:
dview = rc[:]
x = 0
y = 2
dview.push({"x": 0})
handle = rc[0].apply_async(lambda : y + 1)

In [43]:
rc[2].push({"x": 8})
ar = rc[2].apply_async(lambda: x+1)
ar.get_dict()

{2: 9}

In [45]:
import concurrent.futures
def execute_function_multithreaded(exec_command, args_list, max_workers=100):
    """ Execute functions in multithread manner.

    Args:
        exec_command: Callable function to run.
        args_list: The arguments list for exec_command, each element will be passed
            into exec_command and run in multithread paralleled.
        max_workers (int, optional): Defaults to 100.

    Returns:
        A dictionary that maps index to result of exec_command, i.e. index to the
        result of exec_command taking "index"-th args in the args_list.
    """
    results = {}
    _max_workers = min(max_workers, len(args_list))
    with concurrent.futures.ThreadPoolExecutor(max_workers=_max_workers) as executor:
        futures_to_index = {executor.submit(exec_command, arg): index
                            for index, arg in enumerate(args_list)}
        for future in concurrent.futures.as_completed(futures_to_index):
            index = futures_to_index[future]
            results[index] = future.result()
    return results

In [51]:
def incremental(i):
    global x
    for _ in range(100):
        rc[i].push({"x": x})
        ar = rc[i].apply_async(lambda: x+1)
        x = ar.get_dict()[i]
    print(f"{i}: {x}")

In [52]:
x = 0

In [53]:
execute_function_multithreaded(incremental, [0, 1, 2, 3], max_workers=4)

0: 100
1: 100
3: 100
2: 100


{0: None, 1: None, 3: None, 2: None}

In [54]:
x

100

In [55]:
%px print(x)

[stdout:0] 99
[stdout:1] 99
[stdout:2] 99
[stdout:3] 99
